In [1]:
import pickle
import os
import sys
import pandas as pd
import numpy as np
import re

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model ,load_model


from lifelines import CoxPHFitter
from Models.RCFR_AC_RefLong import SetModel
from Module.DataProcessing import DataLoad
from Module.MetricsGroup import DoMetric, DoAggMetric, DoSimEval



os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"



### Data load

In [ ]:
StackedData, IntToGene, TTE, EVENT, TrIndEmbeddMask, ReferencePatIDLong, ReferencePatIDShort, NormDismInd, MergedData= DataLoad()

PatIDX = StackedData[:, 0:1].astype('int')
GeneIDX = StackedData[:, 1:2].astype('int')
GeneExp = StackedData[:, 2:3]

IndN = len(np.unique(PatIDX))
FeatN = len(np.unique(GeneIDX))


## Setting for tasks
#### WeightID indicates theta IDs for ACAM : W0, W10, W20 

In [5]:
# Parameters for post-hoc models
ModelID ='M04'
WeightID = 'W10'

EmbedSize = 50
NCL_Feat = 5
NCL_Ind = 2
AdjCosWeight_ = 1. # This parameter affects only training phases, so thus any float number can be set in the post analysis phases. 
NumGene_CL = 100

ModelList = os.listdir('./Results/')
ModelList = [i for i in ModelList if ModelID in i and WeightID  in i  and 'hdf5' in i and 'NOC' not in i ]
FilePath = './Results/'

# Model structure load
RCFR_AC, LayerList = SetModel(AdjCosWeight_, NormDismInd, TrIndEmbeddMask, IndN, FeatN, ReferencePatIDLong, ReferencePatIDShort)

# Data for calculating metric
DataMetric = [MergedData, TTE, EVENT, NCL_Ind, NCL_Feat, NumGene_CL, IntToGene]

ColList = ['Model','AvgtPRate', 'AvgtAdjPRate', 'MintAdjPRate', 'AvgABSGeCohD', 'MinABSGeCohD', 'AvgABSSurvCoef', 'MinABSSurvCoef', 'AvgSurvpVal', 
           'MaxSurvpVal', 'NegExpAvgSurvpVal', 'NegExpMinSurvpVal', 'AvgNegSigRate',  'MinNegSigRate', 'AvgPosSigRate', 'MinPosSigRate','IndCentRatio']


## Procedure for Selecting best model

In [5]:
MetricTable = pd.DataFrame(columns=ColList)
InfoFeatGroupList = []

for num, model in enumerate(ModelList[:]):
    print(num)
    

    RCFR_AC.load_weights(FilePath + model)  # Model weights load
    InpInd, InpFeat, IndEmbeddWeig, IndEmbeddReferenceLong, FeatEmbeddWeig, IndCentroid, FeatCentroid, ICosCLSim, FCosCLSim = LayerList
        
    # Metric calculation: InfoFeatGroup will be used in UMAP analysis
    metrics, InfoFeatGroup = DoMetric (DataMetric, [InpInd, InpFeat, IndEmbeddWeig, FeatEmbeddWeig, IndCentroid, FeatCentroid, ICosCLSim, FCosCLSim])
    InfoFeatGroupList.append(InfoFeatGroup)
    print(InfoFeatGroup)
    MetricTable = pd.concat([MetricTable, pd.DataFrame([[model] + metrics], columns=ColList)], axis=0)


MetricTable['GroupM'] = np.array([re.findall('.\d+', i)[1][1:] for i in  MetricTable['Model']])
MetricTable['EpNum'] = np.array([ re.findall('.\d+\.', i)[0][1:-1] for i in  MetricTable['Model']]).astype('int')
MetricTable = MetricTable.sort_values(['GroupM','EpNum'])

#MetricTable.to_csv('./MetricTables/RCFR_ACMetricTable_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)

0
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), [0.612, 0.401, 0.615, 0.59]]
1
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.   , 0.002, 0.   , 0.018]), array([0.   , 0.   , 0.   , 0.018]), [0.711, 0.125, 0.616, 0.679]]
2
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.034, 0.75 , 0.024, 0.154]), array([0.   , 0.002, 0.006, 0.118]), [0.504, 0.006, 0.101, 0.006]]
3
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.036, 0.73 , 0.   , 0.176]), array([0.   , 0.002, 0.   , 0.15 ]), [0.005, 0.0, 0.0, 0.0]]
4
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.028, 0.754, 0.   , 0.194]), array([0.   , 0.002, 0.   , 0.186]), [0.001, 0.0, 0.0, 0.0]]
5
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.028, 0.774, 0.004, 0.178]), array([0.  , 0.  , 0.  , 0.21]

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1594: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.496. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  warnings.warn(


0
100
200
0
100
200
0
100
200
[array([0.        , 0.73134328, 0.        , 0.039801  ]), array([0.00497512, 0.        , 0.        , 0.32338308]), [0.0, 0.995, 0.0, 0.0]]
111
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.084, 0.82 , 0.   , 0.154]), array([0.002, 0.002, 0.   , 0.162]), [0.0, 0.0, 0.0, 0.0]]
112
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.07 , 0.84 , 0.036, 0.158]), array([0.002, 0.002, 0.   , 0.166]), [0.0, 0.0, 0.0, 0.0]]
113
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.09 , 0.794, 0.   , 0.142]), array([0.   , 0.   , 0.   , 0.136]), [0.0, 0.0, 0.0, 0.0]]
114
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.11 , 0.874, 0.03 , 0.142]), array([0.  , 0.  , 0.  , 0.14]), [0.0, 0.0, 0.0, 0.0]]
115
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.082, 0.882, 0.07 , 0.152]), array([0.   , 0.   , 0.002, 0

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1601: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  warnings.warn(


100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.02409639, 0.        , 0.        , 0.03815261]), array([0.        , 0.        , 0.14658635, 0.58032129]), [0.851, 0.0, 0.0, 0.0]]
142
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.       , 0.       , 0.       , 0.0465587]), array([0.19433198, 0.00202429, 0.2854251 , 0.6417004 ]), [0.0, 0.0, 0.0, 0.0]]
143
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.        , 0.00208768, 0.        , 0.03757829]), array([0.23173278, 0.27974948, 0.3736952 , 0.70772443]), [0.0, 0.0, 0.0, 0.0]]
144
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.        , 0.        , 0.        , 0.03404255]), array([0.34255319, 0.02553191, 0.31489362, 0.69787234]), [0.0, 0.0, 0.0, 0.0]]
145
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.        , 0.        , 0.01096491, 0.02412281]), array([0.40131579, 0.3

...

0
100
200
0
100
200
0
100
200
[array([0.04526749, 0.6872428 , 0.        , 0.04526749]), array([0.00411523, 0.00411523, 0.        , 0.30041152]), [0.0, 0.995, 0.0, 0.0]]
411
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.086, 0.818, 0.   , 0.154]), array([0.002, 0.002, 0.   , 0.158]), [0.0, 0.0, 0.0, 0.0]]
412
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.046, 0.842, 0.034, 0.156]), array([0.   , 0.002, 0.   , 0.168]), [0.0, 0.0, 0.0, 0.0]]
413
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.084, 0.79 , 0.   , 0.144]), array([0.   , 0.   , 0.   , 0.136]), [0.0, 0.0, 0.0, 0.0]]
414
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.112, 0.872, 0.03 , 0.132]), array([0.   , 0.   , 0.   , 0.136]), [0.0, 0.0, 0.0, 0.0]]
415
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.088, 0.89 , 0.068, 0.156]), array([0.   , 0.   , 0.00

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1718: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / np.array([risk_phi])
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.p

0
100
200
0
100
200
[array([0.01532567, 0.07279693, 0.        , 0.05747126]), array([0.        , 0.        , 0.        , 0.02681992]), [0.0, 0.0, 1.0, 0.0]]
494
0
100
200
0
100
200
0
100
200
0
100
200
[array([0.13409962, 0.42911877, 0.        , 0.07279693]), array([0.00383142, 0.        , 0.        , 0.08429119]), [0.0, 0.0, 0.0, 0.0]]
495
0
100
200
0
100
200
0
100
200
0
100
200
[array([0.01915709, 0.16091954, 0.        , 0.04980843]), array([0.        , 0.        , 0.        , 0.12643678]), [0.043, 0.0, 0.0, 0.0]]
496
0
100
200
0
100
200
0
100
200
0
100
200
[array([0.10505837, 0.53696498, 0.        , 0.05836576]), array([0.        , 0.        , 0.        , 0.23735409]), [0.022, 0.0, 0.0, 0.0]]
497
0
100
200
0
100
200
0
100
200
0
100
200
[array([0.06349206, 0.6468254 , 0.        , 0.04365079]), array([0.        , 0.        , 0.        , 0.28968254]), [0.009, 0.0, 0.0, 0.0]]
498
0
100
200
0
100
200
0
100
200
0
100
200
[array([0.00819672, 0.72131148, 0.        , 0.03278689]), array([0.  

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1594: ConvergenceWarning: Newton-Rhaphson convergence completed successfully but norm(delta) is still high, 0.497. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation in the dataset?

  warnings.warn(


100
200
0
100
200
0
100
200
[array([0.00411523, 0.68312757, 0.        , 0.03292181]), array([0.00411523, 0.00411523, 0.        , 0.28395062]), [0.0, 0.995, 0.0, 0.0]]


## Priority-based filter out by metrics

In [6]:
def Aggregation(MetricTable,AggMetricList):
    AggMetricTable = DoSimEval(MetricTable, 'MaxSurvpVal',pCutoff, AggMetricList, ExcRate, NmodEahG)
    AggMetricRank = DoAggMetric(AggMetricList, AggMetricTable[['Model']+AggMetricList]).sort_values('Metrics')
    AggMetricRank = pd.merge(AggMetricRank, AggMetricTable[['Model','MaxSurvpVal']], on='Model', how='left')
    BestModel = AggMetricRank.sort_values('Metrics').iloc[-1]
    
    return AggMetricRank, BestModel

In [ ]:
pCutoff = 0.005
ExcRate = 0.2
NmodEahG = 1

NegMetricList = ['IndCentRatio', 'MinABSSurvCoef', 'AvgABSSurvCoef',  'MinNegSigRate', 'AvgNegSigRate', 'MinABSGeCohD', 'AvgABSGeCohD']
PosMetricList = ['IndCentRatio', 'MinABSSurvCoef', 'AvgABSSurvCoef', 'MinPosSigRate', 'AvgPosSigRate', 'MinABSGeCohD', 'AvgABSGeCohD']

MetricTable = pd.read_csv('./MetricTables/RCFR_ACMetricTable_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv')
MetricTable['GroupM'] = np.array([re.findall('.\d+', i)[2][1:] for i in  MetricTable['Model']])
MetricTable['EpNum'] = np.array([ re.findall('.\d+\.', i)[0][1:-1] for i in  MetricTable['Model']]).astype('int')




In [6]:
NegAggMetricRank, NegBestModel =  Aggregation(MetricTable, NegMetricList)
PosAggMetricRank, PosBestModel =  Aggregation(MetricTable, PosMetricList)

NegAggMetricRank.to_csv('./MetricTables/RCFR_AC_Neg_AggMetricRank_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)
PosAggMetricRank.to_csv('./MetricTables/RCFR_AC_Pos_AggMetricRank_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)

1
N obs with filter of IndCentRatio : 55
N obs with filter of MinABSSurvCoef : 44
N obs with filter of AvgABSSurvCoef : 36
N obs with filter of MinNegSigRate : 29
N obs with filter of AvgNegSigRate : 24
N obs with filter of MinABSGeCohD : 20
N obs with filter of AvgABSGeCohD : 16

2
N obs with filter of IndCentRatio : 53
N obs with filter of MinABSSurvCoef : 43
N obs with filter of AvgABSSurvCoef : 35
N obs with filter of MinNegSigRate : 28
N obs with filter of AvgNegSigRate : 23
N obs with filter of MinABSGeCohD : 19
N obs with filter of AvgABSGeCohD : 16

3
N obs with filter of IndCentRatio : 56
N obs with filter of MinABSSurvCoef : 45
N obs with filter of AvgABSSurvCoef : 36
N obs with filter of MinNegSigRate : 29
N obs with filter of AvgNegSigRate : 24
N obs with filter of MinABSGeCohD : 20
N obs with filter of AvgABSGeCohD : 16

4
N obs with filter of IndCentRatio : 56
N obs with filter of MinABSSurvCoef : 45
N obs with filter of AvgABSSurvCoef : 36
N obs with filter of MinNegSigR

In [450]:
NegAggMetricRank

,Model,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinNegSigRate,AvgNegSigRate,MinABSGeCohD,AvgABSGeCohD,NormIndCentRatio,NormMinABSSurvCoef,NormAvgABSSurvCoef,NormMinNegSigRate,NormAvgNegSigRate,NormMinABSGeCohD,NormAvgABSGeCohD,Metrics,MaxSurvpVal
0,M04_RCFR_AC_W10_S5_Epo64.hdf5,0.406742,2.683515,3.180491,0.113960,0.381054,0.109298,0.233294,0.594607,0.633491,0.868516,0.000000,0.000000,0.231828,0.000000,2.328442,0.0
1,M04_RCFR_AC_W10_S3_Epo63.hdf5,0.419501,2.488740,3.099534,0.116071,0.404018,0.107945,0.266629,1.000000,0.001058,0.000000,0.074151,0.668014,0.000000,1.000000,2.743223,0.0
2,M04_RCFR_AC_W10_S4_Epo63.hdf5,0.416290,2.488415,3.112668,0.118541,0.404255,0.108574,0.265357,0.897964,0.000000,0.140905,0.160886,0.674921,0.107694,0.961861,2.944231,0.0
3,M04_RCFR_AC_W10_S1_Epo63.hdf5,0.419501,2.677142,3.146631,0.122093,0.404070,0.111102,0.260770,1.000000,0.612800,0.505264,0.285635,0.669524,0.540887,0.824246,4.438355,0.0
4,M04_RCFR_AC_W10_S2_Epo64.hdf5,0.388027,2.796391,3.192747,0.142433,0.415430,0.113781,0.237679,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.131521,5.131521,0.0


In [451]:
PosAggMetricRank

,Model,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinPosSigRate,AvgPosSigRate,MinABSGeCohD,AvgABSGeCohD,NormIndCentRatio,NormMinABSSurvCoef,NormAvgABSSurvCoef,NormMinPosSigRate,NormAvgPosSigRate,NormMinABSGeCohD,NormAvgABSGeCohD,Metrics,MaxSurvpVal
0,M04_RCFR_AC_W10_S2_Epo43.hdf5,0.414966,2.040952,3.491518,0.231733,0.398225,0.111694,0.176246,0.794558,0.716850,1.000000,0.000000,0.006636,0.000000,0.000000,2.518044,0.0
1,M04_RCFR_AC_W10_S5_Epo43.hdf5,0.424658,2.024079,3.376742,0.241164,0.397609,0.114676,0.178023,1.000000,0.637876,0.738767,0.110449,0.000000,0.076785,0.066305,2.630182,0.0
2,M04_RCFR_AC_W10_S3_Epo44.hdf5,0.377483,1.887795,3.052156,0.262156,0.464588,0.150528,0.201265,0.000000,0.000000,0.000000,0.356283,0.721153,1.000000,0.933772,3.011207,0.0
3,M04_RCFR_AC_W10_S4_Epo44.hdf5,0.391304,1.979978,3.235243,0.294949,0.466667,0.145273,0.199311,0.292977,0.431461,0.416711,0.740312,0.743536,0.864692,0.860839,4.350528,0.0
4,M04_RCFR_AC_W10_S1_Epo44.hdf5,0.388982,2.101448,3.268365,0.317125,0.490486,0.150014,0.203040,0.243740,1.000000,0.492097,1.000000,1.000000,0.986760,1.000000,5.722596,0.0
